# SQL TWO
## Análise de Dados com SQL
### Aula 3 - Rank, Dense Rank, Partition By

### 1 - Diretório
#### Uma boa prática é inicialmente limpar o diretório, ou seja, excluir os arquivos (caso já tenham sido criados)

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/nossa_loja

res0: Boolean = true

### 2 - Tabela nossa_loja
#### Vamos criar uma tabela com dados fictícios para nossos tratamentos neste notebook. Utilizaremos DROP e CREATE TABLE para isso.

In [0]:
-- 1 - Criação da base de dados
DROP TABLE IF EXISTS NOSSA_LOJA;

CREATE TABLE NOSSA_LOJA(
  id_estado INT,
  nome_estado VARCHAR(30),
  regiao VARCHAR(30),
  qtd_vendida FLOAT
);

### 3 - Inserindo dados na tabela:
 - INSERT INTO
 - nome da tabela
    - (colunas da tabela)
 - VALUES
    - (dados de cada coluna separados por vírgula)

In [0]:
-- 2 - Inserindo dados na tabela
INSERT INTO
NOSSA_LOJA
VALUES
(1, "São Paulo", "Sudeste", 1800),
(2, "Rio de Janeiro", "Sudeste", 800),
(3, "Minas Gerais", "Sudeste", 2300),
(4, "Espírito Santo", "Sudeste", 1800),
(5, "Paraná", "Sul", 700),
(6, "Santa Catarina", "Sul", 2100),
(7, "Rio Grande do Sul", "Sul", 990),
(8, "Amazonas", "Norte", 4200),
(9, "Pará", "Norte", 3200),
(10, "Acre", "Norte", 1500),
(11, "Bahia", "Nordeste", 1910),
(12, "Pernambuco", "Nordeste", 2380);

num_affected_rows,num_inserted_rows
12,12


### 4 - Visualizando a base de dados

In [0]:
-- 3 - Visualização da tabela
SELECT * FROM NOSSA_LOJA;

id_estado,nome_estado,regiao,qtd_vendida
1,São Paulo,Sudeste,1800.0
2,Rio de Janeiro,Sudeste,800.0
3,Minas Gerais,Sudeste,2300.0
4,Espírito Santo,Sudeste,1800.0
5,Paraná,Sul,700.0
6,Santa Catarina,Sul,2100.0
7,Rio Grande do Sul,Sul,990.0
8,Amazonas,Norte,4200.0
9,Pará,Norte,3200.0
10,Acre,Norte,1500.0


### 5 - Selecionando os dados da região Sudeste

In [0]:
-- 4 - Selecionando os dados da região Sudeste
SELECT
  *
FROM NOSSA_LOJA
WHERE regiao = "Sudeste";

id_estado,nome_estado,regiao,qtd_vendida
1,São Paulo,Sudeste,1800.0
2,Rio de Janeiro,Sudeste,800.0
3,Minas Gerais,Sudeste,2300.0
4,Espírito Santo,Sudeste,1800.0


### 6 - RANK , OVER com ORDER BY
#### Podemos criar um ranking de quantidade vendidas por estado com RANK, OVER com ORDER BY.
- #### O RANK não desempata os empatados: atribui a eles o menor valor (2), e o próximo, mantém sua posição natural (4).

In [0]:
-- 5 - Criando um ranking com as funções RANK e OVER
SELECT
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida,
  RANK() OVER(ORDER BY qtd_vendida DESC) AS ranking
  FROM NOSSA_LOJA
  WHERE regiao = "Sudeste";

id_estado,nome_estado,regiao,qtd_vendida,ranking
3,Minas Gerais,Sudeste,2300.0,1
1,São Paulo,Sudeste,1800.0,2
4,Espírito Santo,Sudeste,1800.0,2
2,Rio de Janeiro,Sudeste,800.0,4


### 7 - DENSE RANK, OVER com ORDER BY
#### Criando um ranking com DENSE RANK, OVER com ORDER BY
- #### O DENSE RANK atribui aos empatados o menor valor (2), e o próximo número natural (3) ao próximo do ranking.

In [0]:
-- 6 - Criando um ranking com as funções DENSE RANK, OVER com ORDER BY
SELECT
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida,
  DENSE_RANK() OVER(ORDER BY qtd_vendida DESC) AS dense_ranking
  FROM NOSSA_LOJA
  WHERE regiao = "Sudeste";

id_estado,nome_estado,regiao,qtd_vendida,dense_ranking
3,Minas Gerais,Sudeste,2300.0,1
1,São Paulo,Sudeste,1800.0,2
4,Espírito Santo,Sudeste,1800.0,2
2,Rio de Janeiro,Sudeste,800.0,3


### 8 - RANK, DENSE RANK, OVER com ORDER BY
#### Podemos verificar a diferença dos 2 tratamentos na tabela olhando especificamente para as linhas seguintes aos empates.

In [0]:
-- 7 - Criando um ranking com RANK, DENSE_RANK, OVER com ORDER BY
SELECT
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida,
  RANK() OVER(ORDER BY qtd_vendida DESC) AS ranking,
  DENSE_RANK() OVER(ORDER BY qtd_vendida DESC) AS dense_ranking
  FROM NOSSA_LOJA;

id_estado,nome_estado,regiao,qtd_vendida,ranking,dense_ranking
8,Amazonas,Norte,4200.0,1,1
9,Pará,Norte,3200.0,2,2
12,Pernambuco,Nordeste,2380.0,3,3
3,Minas Gerais,Sudeste,2300.0,4,4
6,Santa Catarina,Sul,2100.0,5,5
11,Bahia,Nordeste,1910.0,6,6
1,São Paulo,Sudeste,1800.0,7,7
4,Espírito Santo,Sudeste,1800.0,7,7
10,Acre,Norte,1500.0,9,8
7,Rio Grande do Sul,Sul,990.0,10,9


### 9 - RANK, DENSE RANK, OVER + PARTITION BY
#### Podemos criar rankings de acordo com cada particionamento por região com RANK, DENSE_RANK, OVER, PARTITION BY.

In [0]:
-- 8 - Criando um ranking com RANK, DENSE_RANK, OVER com PARTITION BY e ORDER BY
SELECT
  id_estado,
  nome_estado,
  regiao,
  qtd_vendida,
  RANK() OVER(PARTITION BY regiao ORDER BY qtd_vendida DESC) AS ranking,
  DENSE_RANK() OVER(PARTITION BY regiao ORDER BY qtd_vendida DESC) AS dense_ranking
  FROM NOSSA_LOJA;

id_estado,nome_estado,regiao,qtd_vendida,ranking,dense_ranking
12,Pernambuco,Nordeste,2380.0,1,1
11,Bahia,Nordeste,1910.0,2,2
8,Amazonas,Norte,4200.0,1,1
9,Pará,Norte,3200.0,2,2
10,Acre,Norte,1500.0,3,3
3,Minas Gerais,Sudeste,2300.0,1,1
1,São Paulo,Sudeste,1800.0,2,2
4,Espírito Santo,Sudeste,1800.0,2,2
2,Rio de Janeiro,Sudeste,800.0,4,3
6,Santa Catarina,Sul,2100.0,1,1
